In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time

In [2]:
# Obtenemos los ids mirando en la página de fbref
players = {
    'Vinicius Jr': '7111d552',
    'Lewandowski': '8d78e732',
    'Morata': '129af0db',
    'Griezmann': 'df69b544',
    'Nico Williams': 'afdc14d7',
    'Kubo': '16aa3654',
    'Ferran Torres': '9e1035f8',
    'Gerard Moreno': '81f0781e',
    'Miguel Gutiérrez': '7e98cff1',
    'Oscar Mingueza': '056269d7'
}

# Distintas tablas de estadísticas que se pueden obtener de un jugador
tables = ['', 'passing/', 'passing_types/', 'gca/', 'defense/', 'possession/', 'misc/']


def scrap_table(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tabla = soup.find('table', {'id': 'matchlogs_all'}) # Se queda con la tabla de partidos
        datos_tabla = []
        
        # Extraemos los encabezados de la tabla
        fila_encabezados = tabla.find('thead').find_all('tr')[1] # Nos quedamos con la segunda fila de encabezados
        headers = [th.get('aria-label') for th in fila_encabezados.find_all('th')] 
        
        # Extraemos las filas de la tabla
        filas = tabla.select('tbody tr:not([class])') # Evitamos filas separadoras y de partidos que no ha jugado
        for fila in filas:
            celdas = fila.find_all(['th', 'td'])
            datos_fila = [celda.text.strip() for celda in celdas]
            datos_tabla.append(datos_fila)

        return pd.DataFrame(datos_tabla, columns=headers)
    

def get_player_csv(name_player):
    id_player = players[name_player]
    player_link = f'https://fbref.com/es/jugadores/{id_player}/partidos/2023-2024/'
    df = scrap_table(player_link)

    for table in tables:   
        table_link = f'https://fbref.com/es/jugadores/{id_player}/partidos/2023-2024/{table}'
        df_table = scrap_table(table_link)
        df = pd.concat([df, df_table], axis=1)
        time.sleep(5)
    
    df = df.loc[:,~df.columns.duplicated()]
    df.to_csv(f'raw/player_stats/{name_player.replace(' ', '_')}_data.csv', index=False)

In [3]:
def get_player_resume_csv(id_player):
    player_link = f'https://fbref.com/es/jugadores/{id_player}/partidos/2023-2024/'
    response = requests.get(player_link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tabla = soup.find('table', {'id': 'matchlogs_all'}) # Se queda con la tabla de partidos
        datos_tabla = []
        
        # Extraemos los encabezados de la tabla
        fila_encabezados = tabla.find('thead').find_all('tr')[1] # Nos quedamos con la segunda fila de encabezados
        headers = [th.get('aria-label') for th in fila_encabezados.find_all('th')] 
        
        # Extraemos las filas de la tabla
        filas = tabla.select('tbody tr:not([class])') # Evitamos filas separadoras y de partidos que no ha jugado
        for fila in filas:
            celdas = fila.find_all(['th', 'td'])
            datos_fila = [celda.text.strip() for celda in celdas]
            datos_tabla.append(datos_fila)

        # Guardamos los datos en un archivo CSV
        with open(f'raw/{id_player}_data.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)
            writer.writerows(datos_tabla)

In [4]:
# Obtenemos los datos de los jugadores

for player in list(players.keys())[2:]:
    try:
        get_player_csv(player)
        print(f'Obtenidos los datos de {player}')
    except:
        print(f'Error al obtener los datos de {player}. Demasiadas peticiones')
        break

Obtenidos los datos de Morata
Obtenidos los datos de Griezmann
Obtenidos los datos de Nico Williams
Obtenidos los datos de Kubo
Obtenidos los datos de Ferran Torres
Obtenidos los datos de Gerard Moreno
Obtenidos los datos de Miguel Gutiérrez
Obtenidos los datos de Oscar Mingueza


In [7]:
df = pd.read_csv('raw/player_stats/Vinicius_Jr_data.csv')
df.head()

,Fecha,Día,Competencia,Ronda,Sedes,Resultado,Equipo,Adversario,Arranque,Posición,...,Faltas cometidas,Faltas recibidas,Posición adelantada,Penales ejecutados,Penales concedidos,Goles en contra,Recuperación de pelotas,Aéreos Ganados,Aéreos Perdidos,% of Aerials Won
0,2023-08-12,Sáb,La Liga,Semana 1 de partido,Visitante,V 2–0,Real Madrid,Athletic Club,Sí,FW,...,2.0,2.0,2.0,0.0,0.0,0,3.0,0.0,0.0,NaN
1,2023-08-19,Sáb,La Liga,Semana 2 de partido,Visitante,V 3–1,Real Madrid,Almería,Sí,FW,...,2.0,5.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN
2,2023-08-25,Vie,La Liga,Semana 3 de partido,Visitante,V 1–0,Real Madrid,Celta Vigo,Sí,FW,...,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,NaN
3,2023-09-27,Mié,La Liga,Semana 7 de partido,Local,V 2–0,Real Madrid,Las Palmas,No,FW,...,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,NaN
4,2023-09-30,Sáb,La Liga,Semana 8 de partido,Visitante,V 3–0,Real Madrid,Girona,Sí,FW,...,1.0,3.0,0.0,0.0,0.0,0,2.0,0.0,0.0,NaN


In [8]:
for col in df.columns:
    print(col)

Fecha
Día
Competencia
Ronda
Sedes
Resultado
Equipo
Adversario
Arranque
Posición
Minutos
Goles
Asistencias
Tiros penales ejecutados
Tiros penales intentados
Total de disparos
Lanzamientos en el Objetivo
Tarjetas amarillas
Tarjetas rojas
Toques
Derribos
Intercepciones
Bloqueos
xG: Goles esperados
npxG: Goles esperados (xG) sin contar penaltis
xAG: Exp. Objetivos asistidos
Acciones para la creación de tiros
Acciones para la creación de goles
Pases completados
Pases intentados
% de pase completo
Pases progresivos
Transportes
Acarreos progresivos
Tomas intentadas
Tomas exitosas
Informe del partido
Distancia total de pase
Distancia de paso progresiva
Pases completados (Cortos)
Pases intentados (Cortos)
% de pase completo (Cortos)
Pases completados (Medios)
Pases intentados (Medios)
% de pase completo (Medios)
Pases completados (Largos)
Pases intentados (Largos)
% de pase completo (Largos)
xA: Asistencias Esperadas
Pases clave
Pases en el último tercio de la cancha
Pases al área de penalizaci